In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table
import datetime as dt

In [2]:
#Extract csv into dataframes

wage_file = "Data/Minimum Wage Data.csv"
min_wage_df = pd.read_csv(wage_file, low_memory=False, encoding= 'latin')
min_wage_df.head()

,Year,State,State.Minimum.Wage,State.Minimum.Wage.2020.Dollars,Federal.Minimum.Wage,Federal.Minimum.Wage.2020.Dollars,Effective.Minimum.Wage,Effective.Minimum.Wage.2020.Dollars,CPI.Average,Department.Of.Labor.Uncleaned.Data,Department.Of.Labor.Cleaned.Low.Value,Department.Of.Labor.Cleaned.Low.Value.2020.Dollars,Department.Of.Labor.Cleaned.High.Value,Department.Of.Labor.Cleaned.High.Value.2020.Dollars,Footnote
0,1968,Alabama,0.00000,0.00,1.15,8.55,1.15,8.55,34.8,...,0.00000,0.00,0.00000,0.00,NaN
1,1968,Alaska,2.10000,15.61,1.15,8.55,2.10,15.61,34.8,2.1,2.10000,15.61,2.10000,15.61,NaN
2,1968,Arizona,0.46800,3.48,1.15,8.55,1.15,8.55,34.8,18.72 - 26.40/wk(b),0.46800,3.48,0.66000,4.91,(b)
3,1968,Arkansas,0.15625,1.16,1.15,8.55,1.15,8.55,34.8,1.25/day(b),0.15625,1.16,0.15625,1.16,(b)
4,1968,California,1.65000,12.26,1.15,8.55,1.65,12.26,34.8,1.65(b),1.65000,12.26,1.65000,12.26,(b)


In [3]:
min_wage_df.columns

Index(['Year', 'State', 'State.Minimum.Wage',
       'State.Minimum.Wage.2020.Dollars', 'Federal.Minimum.Wage',
       'Federal.Minimum.Wage.2020.Dollars', 'Effective.Minimum.Wage',
       'Effective.Minimum.Wage.2020.Dollars', 'CPI.Average',
       'Department.Of.Labor.Uncleaned.Data',
       'Department.Of.Labor.Cleaned.Low.Value',
       'Department.Of.Labor.Cleaned.Low.Value.2020.Dollars',
       'Department.Of.Labor.Cleaned.High.Value',
       'Department.Of.Labor.Cleaned.High.Value.2020.Dollars', 'Footnote'],
      dtype='object')

In [4]:
# rename columns
min_wage_df = min_wage_df.rename(columns={'State.Minimum.Wage': 'Past_State_Min_Wage', 'State.Minimum.Wage.2020.Dollars':'2020_State_Min_Wage',
                                          'Federal.Minimum.Wage':'Past_Fed_Min_Wage', 'Federal.Minimum.Wage.2020.Dollars':'2020_Fed_Min_Wage',
                                          'Effective.Minimum.Wage':'Past_Eff_Min_Wage','Effective.Minimum.Wage.2020.Dollars':'2020_Eff_Min_Wage',
                                          'CPI.Average': 'CPI_Avg', 'Department.Of.Labor.Cleaned.Low.Value':'DOL_Low', 'Department.Of.Labor.Cleaned.Low.Value.2020.Dollars':
                                          '2020_DOL_Low','Department.Of.Labor.Cleaned.High.Value':'DOL_High','Department.Of.Labor.Cleaned.High.Value.2020.Dollars':'2020_DOL_High'})

In [5]:
#drop columns
min_wage_df.columns
min_wage_df = min_wage_df[['Year', 'State', 'Past_State_Min_Wage', '2020_State_Min_Wage',
       'Past_Fed_Min_Wage', '2020_Fed_Min_Wage', 'Past_Eff_Min_Wage',
       '2020_Eff_Min_Wage', 'CPI_Avg','DOL_Low', '2020_DOL_Low', 'DOL_High', '2020_DOL_High']]
min_wage_df.head()

,Year,State,Past_State_Min_Wage,2020_State_Min_Wage,Past_Fed_Min_Wage,2020_Fed_Min_Wage,Past_Eff_Min_Wage,2020_Eff_Min_Wage,CPI_Avg,DOL_Low,2020_DOL_Low,DOL_High,2020_DOL_High
0,1968,Alabama,0.00000,0.00,1.15,8.55,1.15,8.55,34.8,0.00000,0.00,0.00000,0.00
1,1968,Alaska,2.10000,15.61,1.15,8.55,2.10,15.61,34.8,2.10000,15.61,2.10000,15.61
2,1968,Arizona,0.46800,3.48,1.15,8.55,1.15,8.55,34.8,0.46800,3.48,0.66000,4.91
3,1968,Arkansas,0.15625,1.16,1.15,8.55,1.15,8.55,34.8,0.15625,1.16,0.15625,1.16
4,1968,California,1.65000,12.26,1.15,8.55,1.65,12.26,34.8,1.65000,12.26,1.65000,12.26


In [6]:
#add columns that calculate the percentage of state to federal minimum wage
min_wage_df['Past_Perc_of_Fed'] = round((min_wage_df['Past_State_Min_Wage']/min_wage_df['Past_Fed_Min_Wage']) * 100, 2)
min_wage_df['2020_Perc_of_Fed'] = round((min_wage_df['2020_State_Min_Wage']/min_wage_df['2020_Fed_Min_Wage']) * 100, 2)

In [7]:
# drop columns
min_wage_df.columns
min_wage_df = min_wage_df[['Year', 'State', 'Past_State_Min_Wage', '2020_State_Min_Wage',
       'Past_Fed_Min_Wage', '2020_Fed_Min_Wage', 'Past_Eff_Min_Wage',
       '2020_Eff_Min_Wage', 'CPI_Avg', 'Past_Perc_of_Fed', '2020_Perc_of_Fed']]
min_wage_df.head()

,Year,State,Past_State_Min_Wage,2020_State_Min_Wage,Past_Fed_Min_Wage,2020_Fed_Min_Wage,Past_Eff_Min_Wage,2020_Eff_Min_Wage,CPI_Avg,Past_Perc_of_Fed,2020_Perc_of_Fed
0,1968,Alabama,0.00000,0.00,1.15,8.55,1.15,8.55,34.8,0.00,0.00
1,1968,Alaska,2.10000,15.61,1.15,8.55,2.10,15.61,34.8,182.61,182.57
2,1968,Arizona,0.46800,3.48,1.15,8.55,1.15,8.55,34.8,40.70,40.70
3,1968,Arkansas,0.15625,1.16,1.15,8.55,1.15,8.55,34.8,13.59,13.57
4,1968,California,1.65000,12.26,1.15,8.55,1.65,12.26,34.8,143.48,143.39


In [8]:
#read education level csv
educ_path = 'Data/Education.csv'
educ_df = pd.read_csv(educ_path)
educ_df.head()

,FIPS Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2011-2015","High school diploma only, 2011-2015","Some college or associate's degree, 2011-2015","Bachelor's degree or higher, 2011-2015","Percent of adults with less than a high school diploma, 2011-2015","Percent of adults with a high school diploma only, 2011-2015","Percent of adults completing some college or associate's degree, 2011-2015","Percent of adults with a bachelor's degree or higher, 2011-2015"
0,0,US,United States,NaN,NaN,NaN,NaN,"52,373,312","34,158,051","11,650,730",...,27.4,24.4,"28,229,094","58,722,528","61,558,628","62,952,272",13.3,27.8,29.1,29.8
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,"1,062,306","468,269","136,287",...,25.9,19.0,"509,891","1,005,295","962,515","761,650",15.7,31.0,29.7,23.5
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,"6,611","3,757",933,...,26.9,18.0,"4,656","12,182","11,044","8,437",12.8,33.5,30.4,23.2
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,"18,726","8,426","2,334",...,29.3,23.1,"14,360","39,431","43,500","39,710",10.5,28.8,31.8,29.0
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,"8,120","2,242",581,...,21.3,10.9,"5,021","6,490","4,943","2,354",26.7,34.5,26.3,12.5


In [9]:
#create list of states to eliminate counties
state_list = min_wage_df['State'].unique().tolist()
state_list.append('United States')
state_list

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Guam',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'U.S. Virgin Islands',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming',
 'United States']

In [10]:
#drop rows that are not state info
new_educ_df = educ_df.loc[educ_df['Area name'].isin(state_list)]

In [11]:
#drop columns
new_educ_df = new_educ_df[['State', 'Area name', 'Less than a high school diploma, 1970',
       'High school diploma only, 1970', 'Some college (1-3 years), 1970',
       'Four years of college or higher, 1970',
       'Percent of adults with less than a high school diploma, 1970',
       'Percent of adults with a high school diploma only, 1970',
       'Percent of adults completing some college (1-3 years), 1970',
       'Percent of adults completing four years of college or higher, 1970',
       'Less than a high school diploma, 1980',
       'High school diploma only, 1980', 'Some college (1-3 years), 1980',
       'Four years of college or higher, 1980',
       'Percent of adults with less than a high school diploma, 1980',
       'Percent of adults with a high school diploma only, 1980',
       'Percent of adults completing some college (1-3 years), 1980',
       'Percent of adults completing four years of college or higher, 1980',
       'Less than a high school diploma, 1990',
       'High school diploma only, 1990',
       "Some college or associate's degree, 1990",
       "Bachelor's degree or higher, 1990",
       'Percent of adults with less than a high school diploma, 1990',
       'Percent of adults with a high school diploma only, 1990',
       "Percent of adults completing some college or associate's degree, 1990",
       "Percent of adults with a bachelor's degree or higher, 1990",
       "Less than a high school diploma, 2000",
       'High school diploma only, 2000',
       "Some college or associate's degree, 2000",
       "Bachelor's degree or higher, 2000",
       'Percent of adults with less than a high school diploma, 2000',
       'Percent of adults with a high school diploma only, 2000',
       "Percent of adults completing some college or associate's degree, 2000",
       "Percent of adults with a bachelor's degree or higher, 2000",
       'Less than a high school diploma, 2011-2015',
       "High school diploma only, 2011-2015",
       "Some college or associate's degree, 2011-2015",
       "Bachelor's degree or higher, 2011-2015",
       'Percent of adults with less than a high school diploma, 2011-2015',
       'Percent of adults with a high school diploma only, 2011-2015',
       "Percent of adults completing some college or associate's degree, 2011-2015",
       "Percent of adults with a bachelor's degree or higher, 2011-2015"]]

In [12]:
#rename columns
new_educ_df = new_educ_df.rename(columns ={'Less than a high school diploma, 1970':'1970 < high school',
       'High school diploma only, 1970':'1970 high school', 'Some college (1-3 years), 1970': '1970 some college',
       'Four years of college or higher, 1970':'1970 college > 4',
       'Percent of adults with less than a high school diploma, 1970':'1970 percent < high school',
       'Percent of adults with a high school diploma only, 1970':'1970 percent high school',
       'Percent of adults completing some college (1-3 years), 1970': '1970 percent some college',
       'Percent of adults completing four years of college or higher, 1970':'1970 percent college > 4',
       'Less than a high school diploma, 1980':'1980 <high school',
       'High school diploma only, 1980': '1980 high school', 'Some college (1-3 years), 1980': '1980 some college',
       'Four years of college or higher, 1980': '1980 college > 4',
       'Percent of adults with less than a high school diploma, 1980': '1980 percent < high school',
       'Percent of adults with a high school diploma only, 1980': '1980 percent high school',
       'Percent of adults completing some college (1-3 years), 1980':'1980 percent some college',
       'Percent of adults completing four years of college or higher, 1980':'1980 percent college > 4',
       'Less than a high school diploma, 1990': '1990 < high school',
       'High school diploma only, 1990':'1990 high school',
       "Some college or associate's degree, 1990":'1990 some college',
       "Bachelor's degree or higher, 1990": '1990 college > 4',
       'Percent of adults with less than a high school diploma, 1990':'1990 percent < high school',
       'Percent of adults with a high school diploma only, 1990':'1990 percent high school',
       "Percent of adults completing some college or associate's degree, 1990":'1990 percent some college',
       "Percent of adults with a bachelor's degree or higher, 1990":'1990 percent college > 4',
       "Less than a high school diploma, 2000":'2000 < high school',
       'High school diploma only, 2000':'2000 high school',
       "Some college or associate's degree, 2000":'2000 some college',
       "Bachelor's degree or higher, 2000": '2000 college > 4',
       'Percent of adults with less than a high school diploma, 2000': '2000 percent < high school',
       'Percent of adults with a high school diploma only, 2000':'2000 percent high school',
       "Percent of adults completing some college or associate's degree, 2000":'2000 percent some college',
       "Percent of adults with a bachelor's degree or higher, 2000":'2000 percent college > 4',
       'Less than a high school diploma, 2011-2015':'2011-2015 some high school',
       "High school diploma only, 2011-2015":'2011-2015 high school',
       "Some college or associate's degree, 2011-2015":'2011-2015 some college',
       "Bachelor's degree or higher, 2011-2015":'2011-2015 college > 4',
       'Percent of adults with less than a high school diploma, 2011-2015':'2011-2015 percent < high school',
       'Percent of adults with a high school diploma only, 2011-2015': '2011-2015 percent high school',
       "Percent of adults completing some college or associate's degree, 2011-2015": '2011-2015 percent some college',
       "Percent of adults with a bachelor's degree or higher, 2011-2015":'2011-2015 percent college > 4'})

In [26]:
new_educ_df.reset_index(drop=True, inplace=True)
new_educ_df

,State,Area name,1970 < high school,1970 high school,1970 some college,1970 college > 4,1970 percent < high school,1970 percent high school,1970 percent some college,1970 percent college > 4,...,2000 percent some college,2000 percent college > 4,2011-2015 some high school,2011-2015 high school,2011-2015 some college,2011-2015 college > 4,2011-2015 percent < high school,2011-2015 percent high school,2011-2015 percent some college,2011-2015 percent college > 4
0,US,United States,"52,373,312","34,158,051","11,650,730","11,717,266",47.7,31.1,10.6,10.7,...,27.4,24.4,"28,229,094","58,722,528","61,558,628","62,952,272",13.3,27.8,29.1,29.8
1,AL,Alabama,"1,062,306","468,269","136,287","141,936",58.7,25.9,7.5,7.8,...,25.9,19.0,"509,891","1,005,295","962,515","761,650",15.7,31.0,29.7,23.5
2,AK,Alaska,"44,994","50,820","20,052","19,082",33.3,37.7,14.9,14.1,...,35.7,24.7,"36,493","129,324","168,702","129,921",7.9,27.8,36.3,28.0
3,AZ,Arizona,"383,873","289,781","127,063","115,020",41.9,31.6,13.9,12.6,...,33.1,23.5,"609,245","1,063,765","1,489,284","1,199,010",14.0,24.4,34.1,27.5
4,AR,Arkansas,"635,149","271,150","80,835","70,378",60.1,25.6,7.6,6.7,...,24.5,16.7,"299,099","685,277","564,266","414,099",15.2,34.9,28.7,21.1
5,CA,California,"4,062,367","3,566,578","1,784,772","1,462,266",37.4,32.8,16.4,13.4,...,30.0,26.6,"4,599,641","5,231,824","7,487,209","7,939,184",18.2,20.7,29.6,31.4
6,CO,Colorado,"411,837","392,787","166,188","170,326",36.1,34.4,14.6,14.9,...,31.0,32.7,"327,222","767,325","1,083,623","1,342,007",9.3,21.8,30.8,38.1
7,CT,Connecticut,"740,830","534,830","179,635","230,303",44.0,31.7,10.7,13.7,...,24.1,31.4,"249,857","673,973","613,418","925,607",10.1,27.4,24.9,37.6
8,DE,Delaware,"130,463","91,211","28,091","37,630",45.4,31.7,9.8,13.1,...,26.1,25.0,"73,373","196,236","172,147","189,362",11.6,31.1,27.3,30.0
9,DC,District of Columbia,"189,720","110,767","47,331","75,233",44.8,26.2,11.2,17.8,...,18.2,39.1,"48,430","81,697","76,073","247,752",10.7,18.0,16.8,54.6


In [14]:
#read in unemployment csv
employ_path = 'Data/USUnemployment.csv'
employ_df = pd.read_csv(employ_path)
employ_df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7


In [15]:
#add column to calculate average of unemployment rates
employ_df['Yearly Average']=round((employ_df['Jan']+employ_df['Feb']+employ_df['Mar']+employ_df['Apr']+employ_df['May']+employ_df['Jun']+employ_df['Jul']+employ_df['Aug']+
                    employ_df['Sep']+employ_df['Oct']+employ_df['Nov']+employ_df['Dec'])/12, 2)
employ_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Yearly Average
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0,3.75
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6,6.05
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3,5.21
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1,3.28
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7,3.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2015,5.7,5.5,5.4,5.4,5.6,5.3,5.2,5.1,5.0,5.0,5.1,5.0,5.28
68,2016,4.9,4.9,5.0,5.0,4.8,4.9,4.8,4.9,5.0,4.9,4.7,4.7,4.88
69,2017,4.7,4.6,4.4,4.4,4.4,4.3,4.3,4.4,4.2,4.1,4.2,4.1,4.34
70,2018,4.1,4.1,4.0,4.0,3.8,4.0,3.8,3.8,3.7,3.8,3.7,3.9,3.89


In [27]:
# narrow down years to match minimum wage data
employ_df= employ_df.loc[employ_df['Year'] >= 1968]
employ_df.reset_index(drop=True, inplace=True)
employ_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Yearly Average
0,1968,3.7,3.8,3.7,3.5,3.5,3.7,3.7,3.5,3.4,3.4,3.4,3.4,3.56
1,1969,3.4,3.4,3.4,3.4,3.4,3.5,3.5,3.5,3.7,3.7,3.5,3.5,3.49
2,1970,3.9,4.2,4.4,4.6,4.8,4.9,5.0,5.1,5.4,5.5,5.9,6.1,4.98
3,1971,5.9,5.9,6.0,5.9,5.9,5.9,6.0,6.1,6.0,5.8,6.0,6.0,5.95
4,1972,5.8,5.7,5.8,5.7,5.7,5.7,5.6,5.6,5.5,5.6,5.3,5.2,5.60
5,1973,4.9,5.0,4.9,5.0,4.9,4.9,4.8,4.8,4.8,4.6,4.8,4.9,4.86
6,1974,5.1,5.2,5.1,5.1,5.1,5.4,5.5,5.5,5.9,6.0,6.6,7.2,5.64
7,1975,8.1,8.1,8.6,8.8,9.0,8.8,8.6,8.4,8.4,8.4,8.3,8.2,8.48
8,1976,7.9,7.7,7.6,7.7,7.4,7.6,7.8,7.8,7.6,7.7,7.8,7.8,7.70
9,1977,7.5,7.6,7.4,7.2,7.0,7.2,6.9,7.0,6.8,6.8,6.8,6.4,7.05


In [17]:
#read gdp csv
gdp_path = 'Data/GDP.csv'
gdp_df = pd.read_csv(gdp_path)
gdp_df.head()

,DATE,GDP
0,1947-01-01,243.164
1,1947-04-01,245.968
2,1947-07-01,249.585
3,1947-10-01,259.745
4,1948-01-01,265.742


In [18]:
#parse column to datetime
gdp_df['DATE']=pd.to_datetime(gdp_df['DATE'], format='%Y-%m-%d')

In [19]:
#pull out year and then group by to find average of each year
gdp_df['YEAR']=gdp_df.DATE.dt.year
grouped_df=gdp_df.groupby('YEAR')
avg_gdp = grouped_df.mean()
merged_gdp_df = gdp_df.merge(avg_gdp, on='YEAR')

In [28]:
#rename columns
merged_gdp_df=merged_gdp_df.rename(columns={'GDP_x':'Quarterly GDP', 'GDP_y': 'AVG yearly GDP'})
#drop rows before 1968 to match other datasets
merged_gdp_df = merged_gdp_df.loc[merged_gdp_df['YEAR']>=1968]
merged_gdp_df.reset_index(drop=True, inplace=True)

In [2]:
from config import connection_string, engine   



In [30]:
#add dataaframe to table
min_wage_df.to_sql(name='min_wage', con=engine, if_exists='replace', index=True)
new_educ_df.to_sql(name='educ_level', con=engine, if_exists='replace', index=True)
employ_df.to_sql(name='unemployment', con=engine, if_exists='replace', index=True)
merged_gdp_df.to_sql(name='gdp', con=engine, if_exists='replace', index=True)

In [31]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)

In [32]:

# Confirm tables
engine.table_names()


['min_wage', 'educ_level', 'unemployment', 'gdp']